In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
import geopandas as gpd
import pandas as pd

# 1️⃣ Point to your GeoJSONs and target CRS for Lausanne
INPUTS = {
    1721 : "src/data/Melotte-1721.geojson",
    1831: "src/data/Berney_espaces_verts_macro.geojson",
    1888: "src/data/espaces_verts_macro.geojson",
    2024: "src/data/2024_espaces_verts_macro.geojson",
}
TARGET_CRS = "EPSG:2056"  # Swiss grid, good for metres

# 2️⃣ Collect raw per-feature areas
records = []
for year, path in INPUTS.items():
    print(f"Processing {year} → {path}")
    gdf = gpd.read_file(path)

    # si pas de CRS, on assume WGS84
    if gdf.crs is None:
        print(f"⚠️ CRS non défini pour « {path} », j'assume EPSG:4326")
        gdf = gdf.set_crs("EPSG:4326")

    # reprojection en Swiss grid
    gdf = gdf.to_crs(TARGET_CRS)

    # calcul des surfaces
    gdf["area_m2"] = gdf.geometry.area
    # conversion en hectares
    gdf["area_ha"] = gdf["area_m2"] / 10_000

    # extraire les attributs
    for _, row in gdf.iterrows():
        records.append({
            "year":     year,
            "merge_id": row.get("merge_id", ""),
            "class":    row["macro_ev"],
            "area_m2":  row["area_m2"],
            "area_ha":  row["area_ha"],
        })

# 3️⃣ Écriture du tableau brut (une ligne par polygone)
df = pd.DataFrame(records)
df.to_csv("src/data/areas_raw.csv", index=False)

# 4️⃣ Agrégation par année + classe (en hectares)
summary = (
    df
    .groupby(["year", "class"], as_index=False)
    .agg(
         total_area_ha = ("area_ha", "sum"))
)
summary.to_csv("src/data/areas_by_class.csv", index=False)

print("Wrote src/data/areas_raw.csv and src/data/areas_by_class.csv")


Processing 1721 → src/data/Melotte-1721.geojson


Skipping field own_col_de: unsupported OGR type: 5


Processing 1831 → src/data/Berney_espaces_verts_macro.geojson
Processing 1888 → src/data/espaces_verts_macro.geojson
Processing 2024 → src/data/2024_espaces_verts_macro.geojson
Wrote src/data/areas_raw.csv and src/data/areas_by_class.csv
